# 구글드라이브 연결

In [4]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

path = "/content/gdrive/MyDrive/lecture/"
os.listdir(path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


['무비렌즈 데이터 전처리.ipynb',
 'movies.csv',
 'ratings.csv',
 'tags.csv',
 'links.csv',
 'movies_refined.csv',
 'ratings_refined.csv',
 '협업 필터링 - User-based.ipynb',
 'movie_user.csv',
 'movie_user.gsheet',
 '제목 없는 문서.gdoc',
 '협업 필터링 - Item-based.ipynb',
 '협업 필터링 - User-based with dot production.ipynb']

# 데이터 로드

https://grouplens.org/datasets/movielens/

https://files.grouplens.org/datasets/movielens/ml-10m-README.html

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd
import numpy as np

In [8]:
movies = pd.read_csv("/content/drive/MyDrive/lecture/movies.csv")
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [10]:
ratings = pd.read_csv("/content/drive/MyDrive/lecture/ratings.csv")
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [11]:
tags = pd.read_csv("/content/drive/MyDrive/lecture/tags.csv")
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


# 영화 데이터

## 중복체크

In [12]:
len(movies)  # 무비 데이터 수

9742

In [13]:
movies['movieId'].nunique()  # 무비 id 수

9742

In [14]:
movies['title'].nunique()  # 무비 제목 != 무비 id 수

9737

In [15]:
movies[movies.duplicated('title')]

,movieId,title,genres
5601,26958,Emma (1996),Romance
6932,64997,War of the Worlds (2005),Action|Sci-Fi
9106,144606,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Romance|Thriller
9135,147002,Eros (2004),Drama|Romance
9468,168358,Saturn 3 (1980),Sci-Fi|Thriller


In [16]:
movies['title'].value_counts()[movies['title'].value_counts() > 1]

Emma (1996)                               2
War of the Worlds (2005)                  2
Confessions of a Dangerous Mind (2002)    2
Eros (2004)                               2
Saturn 3 (1980)                           2
Name: title, dtype: int64

## 중복값 처리

영화1) Emma

movies 데이터 처리

In [17]:
movies[movies['title'] == 'Emma (1996)']

,movieId,title,genres
650,838,Emma (1996),Comedy|Drama|Romance
5601,26958,Emma (1996),Romance


In [18]:
movies = movies[~(movies['movieId'] == 26958)].copy()
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


ratings 데이터 처리

In [19]:
ratings[ratings['movieId'] == 26958]

,userId,movieId,rating,timestamp
80596,509,26958,3.5,1436031753


In [20]:
ratings.loc[80596, 'movieId'] = 838

In [21]:
ratings[ratings['movieId'] == 26958]

,userId,movieId,rating,timestamp


In [22]:
ratings.loc[[80596]]

,userId,movieId,rating,timestamp
80596,509,838,3.5,1436031753


영화2) War of the Worlds

movies 데이터 처리

In [23]:
movies[movies['title'] == 'War of the Worlds (2005)']

,movieId,title,genres
5931,34048,War of the Worlds (2005),Action|Adventure|Sci-Fi|Thriller
6932,64997,War of the Worlds (2005),Action|Sci-Fi


In [24]:
movies = movies[~(movies['movieId'] == 64997)].copy()
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


ratings 데이터 처리

In [25]:
ratings[ratings['movieId'] == 64997]

,userId,movieId,rating,timestamp
4747,28,64997,3.5,1234850075
11451,68,64997,2.5,1230497715


In [26]:
ratings.loc[4747, 'movieId'] = 34048
ratings.loc[11451, 'movieId'] = 34048

In [27]:
ratings[ratings['movieId'] == 64997]

,userId,movieId,rating,timestamp


In [28]:
ratings.loc[[4747, 11451]]

,userId,movieId,rating,timestamp
4747,28,34048,3.5,1234850075
11451,68,34048,2.5,1230497715


영화3) Confessions of a Dangerous Mind

movies 데이터 처리

In [29]:
movies[movies['title'] == 'Confessions of a Dangerous Mind (2002)']

,movieId,title,genres
4169,6003,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Thriller
9106,144606,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Romance|Thriller


In [30]:
movies.loc[4169, 'genres'] = 'Comedy|Crime|Drama|Romance|Thriller'
movies[movies['title'] == 'Confessions of a Dangerous Mind (2002)']

,movieId,title,genres
4169,6003,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Romance|Thriller
9106,144606,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Romance|Thriller


In [31]:
movies = movies[~(movies['movieId'] == 144606)].copy()
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


ratings 데이터 처리

In [32]:
ratings[ratings['movieId'] == 144606]

,userId,movieId,rating,timestamp
17819,111,144606,4.0,1517441257


In [33]:
ratings.loc[17819, 'movieId'] = 6003

In [34]:
ratings[ratings['movieId'] == 144606]

,userId,movieId,rating,timestamp


In [35]:
ratings.loc[[17819]]

,userId,movieId,rating,timestamp
17819,111,6003,4.0,1517441257


영화4) Eros

movie 데이터 처리

In [36]:
movies[movies['title'] == 'Eros (2004)']

,movieId,title,genres
5854,32600,Eros (2004),Drama
9135,147002,Eros (2004),Drama|Romance


In [37]:
movies.loc[5854, 'genres'] = 'Drama|Romance'
movies[movies['title'] == 'Eros (2004)']

,movieId,title,genres
5854,32600,Eros (2004),Drama|Romance
9135,147002,Eros (2004),Drama|Romance


In [38]:
movies = movies[~(movies['movieId'] == 147002)].copy()
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


ratings 데이터 처리

In [39]:
ratings[ratings['movieId'] == 147002]

,userId,movieId,rating,timestamp
49832,318,147002,4.0,1502207152


In [40]:
ratings.loc[49832, 'movieId'] = 32600

In [41]:
ratings[ratings['movieId'] == 147002]

,userId,movieId,rating,timestamp


In [42]:
ratings.loc[[49832]]

,userId,movieId,rating,timestamp
49832,318,32600,4.0,1502207152


영화5) Saturn 3

In [43]:
movies[movies['title'] == 'Saturn 3 (1980)']

,movieId,title,genres
2141,2851,Saturn 3 (1980),Adventure|Sci-Fi|Thriller
9468,168358,Saturn 3 (1980),Sci-Fi|Thriller


In [44]:
movies = movies[~(movies['movieId'] == 168358)].copy()
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


ratings 데이터 처리

In [45]:
ratings[ratings['movieId'] == 168358]

,userId,movieId,rating,timestamp
81458,514,168358,2.5,1533945970


In [46]:
ratings.loc[81458, 'movieId'] = 2851

In [47]:
ratings[ratings['movieId'] == 168358]

,userId,movieId,rating,timestamp


In [48]:
ratings.loc[[81458]]

,userId,movieId,rating,timestamp
81458,514,2851,2.5,1533945970


처리 확인

In [49]:
len(movies)

9737

In [50]:
movies['movieId'].nunique()

9737

In [51]:
movies['title'].nunique()

9737

## 장르 결측치 처리

Imputation

In [52]:
no_genre_cond = movies['genres'].str.contains(' ')
movies[no_genre_cond]

,movieId,title,genres
8517,114335,La cravate (1957),(no genres listed)
8684,122888,Ben-hur (2016),(no genres listed)
8687,122896,Pirates of the Caribbean: Dead Men Tell No Tal...,(no genres listed)
8782,129250,Superfast! (2015),(no genres listed)
8836,132084,Let It Be Me (1995),(no genres listed)
8902,134861,Trevor Noah: African American (2013),(no genres listed)
9033,141131,Guardians (2016),(no genres listed)
9053,141866,Green Room (2015),(no genres listed)
9070,142456,The Brand New Testament (2015),(no genres listed)
9091,143410,Hyena Road,(no genres listed)


In [53]:
len(movies[no_genre_cond])

34

In [54]:
len(movies[~no_genre_cond])

9703

In [55]:
del_movieId = movies.loc[no_genre_cond, 'movieId'].tolist()
len(del_movieId)

34

In [56]:
movies = movies[~movies['movieId'].isin(del_movieId)].copy()
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [57]:
movies[no_genre_cond]

<ipython-input-57-f40535573ce0>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  movies[no_genre_cond]


,movieId,title,genres


ratings 데이터도 처리 필요

In [58]:
len(ratings)

100836

In [59]:
# 삭제할 영화 관련 데이터
len(ratings[ratings['movieId'].isin(del_movieId)])

47

In [60]:
100836 - 47  # 삭제 후 데이터 개수

100789

In [61]:
ratings = ratings[~ratings['movieId'].isin(del_movieId)].copy()
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


## 그외 결측치 체크

In [62]:
movies.columns[movies.isna().any()].tolist()

[]

## 인덱스 정리

In [63]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [64]:
movies = movies.reset_index(drop=True)
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9698,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9699,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9700,193585,Flint (2017),Drama
9701,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [65]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [66]:
ratings = ratings.reset_index(drop=True)
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100784,610,166534,4.0,1493848402
100785,610,168248,5.0,1493850091
100786,610,168250,5.0,1494273047
100787,610,168252,5.0,1493846352


## 구분자 변경

TfidfVectorizer 사용하려면 space 를 seperator 로 써야

In [67]:
# 장르 이름에는 빈칸 없다
# 만약 빈칸 있으면 _ 붙이는 처리 필요
# 예) k drama|fun --> k drama fun  (X)
#                 --> k_drama fun  (O)
movies[movies['genres'].str.contains(' ')]

,movieId,title,genres


In [68]:
movies['genres'] = movies['genres'].str.replace('|',' ', regex=True)
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9698,193581,Black Butler: Book of the Atlantic (2017),Action Animation Comedy Fantasy
9699,193583,No Game No Life: Zero (2017),Animation Comedy Fantasy
9700,193585,Flint (2017),Drama
9701,193587,Bungo Stray Dogs: Dead Apple (2018),Action Animation


## 연도 결측치 처리

In [69]:
# 제목에 연도 없는 데이터 검색
# movies[~movies['title'].str.contains('(')]  <-- \( 사용해야함
movies[~movies['title'].str.contains('\(')]

,movieId,title,genres
6058,40697,Babylon 5,Sci-Fi
9023,140956,Ready Player One,Action Sci-Fi Thriller
9163,149334,Nocturnal Animals,Drama Thriller
9345,162414,Moonlight,Drama


In [ ]:
# 'Babylon 5': 1994
# 'Ready Player One': 2018
# 'Nocturnal Animals': 2017
# 'Moonlight': 2009

In [70]:
tmp = movies.copy()

In [71]:
tmp.loc[6058, 'title'] = tmp.loc[6058, 'title'] + " (1994)"
tmp.loc[[6058]]

,movieId,title,genres
6058,40697,Babylon 5 (1994),Sci-Fi


In [72]:
# 중복 실행 시 연도 중첩 방지 필요
tmp.loc[6058, 'title'] = tmp.loc[6058, 'title'] + " (1994)"
tmp.loc[[6058]]

,movieId,title,genres
6058,40697,Babylon 5 (1994) (1994),Sci-Fi


In [73]:
# 연도 없는지 체크
'(' not in tmp.loc[6058, 'title']

False

In [74]:
tmp = movies.copy()
idx = 6058
year = 1994

if '(' not in tmp.loc[idx, 'title']:
    tmp.loc[idx, 'title'] = tmp.loc[idx, 'title'] + f" ({year})"

tmp.loc[[idx]]

,movieId,title,genres
6058,40697,Babylon 5 (1994),Sci-Fi


In [75]:
# 직접 입력
idx = 6058
year = 1994
if '(' not in movies.loc[idx, 'title']:
    movies.loc[idx, 'title'] = movies.loc[idx, 'title'] + f" ({year})"
movies.loc[[idx]]

,movieId,title,genres
6058,40697,Babylon 5 (1994),Sci-Fi


In [76]:
# 직접 입력
idx = 9023
year = 2018
if '(' not in movies.loc[idx, 'title']:
    movies.loc[idx, 'title'] = movies.loc[idx, 'title'] + f" ({year})"
movies.loc[[idx]]

,movieId,title,genres
9023,140956,Ready Player One (2018),Action Sci-Fi Thriller


In [77]:
# 직접 입력
idx = 9163
year = 2017
if '(' not in movies.loc[idx, 'title']:
    movies.loc[idx, 'title'] = movies.loc[idx, 'title'] + f" ({year})"
movies.loc[[idx]]

,movieId,title,genres
9163,149334,Nocturnal Animals (2017),Drama Thriller


In [78]:
# 직접 입력
idx = 9345
year = 2009
if '(' not in movies.loc[idx, 'title']:
    movies.loc[idx, 'title'] = movies.loc[idx, 'title'] + f" ({year})"
movies.loc[[idx]]

,movieId,title,genres
9345,162414,Moonlight (2009),Drama


In [79]:
# 제목에 연도 없는 데이터 검색
movies[~movies['title'].str.contains('\(')]

,movieId,title,genres


## 영화명/연도 분리

### 연습

In [80]:
"Waiting to Exhale (1995)".rsplit(' ', 1) #기본 split은 왼쪽에서부터 실행 우리는 연도에 대해 split을 사용해야하므로 rsplit으로 작성

['Waiting to Exhale', '(1995)']

In [81]:
# 여백 주의! split에 영향줌
"Waiting to Exhale (1995) ".rsplit(' ', 1)

['Waiting to Exhale (1995)', '']

In [82]:
"Waiting to Exhale (1995) ".strip()

'Waiting to Exhale (1995)'

In [83]:
"Waiting to Exhale (1995) ".strip().rsplit(' ', 1)

['Waiting to Exhale', '(1995)']

In [84]:
movies['title'].str.rsplit(' ', n=1)

0                                [Toy Story, (1995)]
1                                  [Jumanji, (1995)]
2                         [Grumpier Old Men, (1995)]
3                        [Waiting to Exhale, (1995)]
4              [Father of the Bride Part II, (1995)]
                            ...                     
9698    [Black Butler: Book of the Atlantic, (2017)]
9699                 [No Game No Life: Zero, (2017)]
9700                                 [Flint, (2017)]
9701          [Bungo Stray Dogs: Dead Apple, (2018)]
9702          [Andrew Dice Clay: Dice Rules, (1991)]
Name: title, Length: 9703, dtype: object

In [85]:
movies['title'].str.rsplit(' ', n=1, expand=True)

,0,1
0,Toy Story,(1995)
1,Jumanji,(1995)
2,Grumpier Old Men,(1995)
3,Waiting to Exhale,(1995)
4,Father of the Bride Part II,(1995)
...,...,...
9698,Black Butler: Book of the Atlantic,(2017)
9699,No Game No Life: Zero,(2017)
9700,Flint,(2017)
9701,Bungo Stray Dogs: Dead Apple,(2018)


### 적용

In [86]:
movies['title'] = movies['title'].str.strip()
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9698,193581,Black Butler: Book of the Atlantic (2017),Action Animation Comedy Fantasy
9699,193583,No Game No Life: Zero (2017),Animation Comedy Fantasy
9700,193585,Flint (2017),Drama
9701,193587,Bungo Stray Dogs: Dead Apple (2018),Action Animation


In [87]:
movies[['title', 'year']] = movies['title'].str.rsplit(' ', n=1, expand=True)
movies

,movieId,title,genres,year
0,1,Toy Story,Adventure Animation Children Comedy Fantasy,(1995)
1,2,Jumanji,Adventure Children Fantasy,(1995)
2,3,Grumpier Old Men,Comedy Romance,(1995)
3,4,Waiting to Exhale,Comedy Drama Romance,(1995)
4,5,Father of the Bride Part II,Comedy,(1995)
...,...,...,...,...
9698,193581,Black Butler: Book of the Atlantic,Action Animation Comedy Fantasy,(2017)
9699,193583,No Game No Life: Zero,Animation Comedy Fantasy,(2017)
9700,193585,Flint,Drama,(2017)
9701,193587,Bungo Stray Dogs: Dead Apple,Action Animation,(2018)


In [88]:
movies['year'] = movies['year'].apply(lambda x: x[1:-1])
movies['year'] = movies['year'].astype(int)
movies

,movieId,title,genres,year
0,1,Toy Story,Adventure Animation Children Comedy Fantasy,1995
1,2,Jumanji,Adventure Children Fantasy,1995
2,3,Grumpier Old Men,Comedy Romance,1995
3,4,Waiting to Exhale,Comedy Drama Romance,1995
4,5,Father of the Bride Part II,Comedy,1995
...,...,...,...,...
9698,193581,Black Butler: Book of the Atlantic,Action Animation Comedy Fantasy,2017
9699,193583,No Game No Life: Zero,Animation Comedy Fantasy,2017
9700,193585,Flint,Drama,2017
9701,193587,Bungo Stray Dogs: Dead Apple,Action Animation,2018


In [89]:
movies = movies[['movieId', 'title', 'year', 'genres']]
movies

,movieId,title,year,genres
0,1,Toy Story,1995,Adventure Animation Children Comedy Fantasy
1,2,Jumanji,1995,Adventure Children Fantasy
2,3,Grumpier Old Men,1995,Comedy Romance
3,4,Waiting to Exhale,1995,Comedy Drama Romance
4,5,Father of the Bride Part II,1995,Comedy
...,...,...,...,...
9698,193581,Black Butler: Book of the Atlantic,2017,Action Animation Comedy Fantasy
9699,193583,No Game No Life: Zero,2017,Animation Comedy Fantasy
9700,193585,Flint,2017,Drama
9701,193587,Bungo Stray Dogs: Dead Apple,2018,Action Animation


# 데이터 저장

In [91]:
movies.to_csv('/content/drive/MyDrive/lecture/movies_refined.csv', index=False)
movies

,movieId,title,year,genres
0,1,Toy Story,1995,Adventure Animation Children Comedy Fantasy
1,2,Jumanji,1995,Adventure Children Fantasy
2,3,Grumpier Old Men,1995,Comedy Romance
3,4,Waiting to Exhale,1995,Comedy Drama Romance
4,5,Father of the Bride Part II,1995,Comedy
...,...,...,...,...
9698,193581,Black Butler: Book of the Atlantic,2017,Action Animation Comedy Fantasy
9699,193583,No Game No Life: Zero,2017,Animation Comedy Fantasy
9700,193585,Flint,2017,Drama
9701,193587,Bungo Stray Dogs: Dead Apple,2018,Action Animation


In [95]:
ratings.to_csv('/content/drive/MyDrive/lecture/ratings_refined.csv', index=False)
ratings

#from datatime을 연도로 바꿔주는 함수 (timestamp)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100784,610,166534,4.0,1493848402
100785,610,168248,5.0,1493850091
100786,610,168250,5.0,1494273047
100787,610,168252,5.0,1493846352


In [97]:
timestamp.fromtimestamp(964982703)

NameError: ignored

In [ ]:
time_unit = 2
datetime.fromtimestamp(964982703 / time_unit)

datetime.datetime(1985, 4, 16, 9, 22, 31, 500000)

In [ ]:
from datetime import datetime
tmp = ratings.copy()
time_unit = 2
tmp['datetime'] = tmp['timestamp'].apply(lambda x: datetime.fromtimestamp(x/time_unit))
tmp

,userId,movieId,rating,timestamp,datetime
0,1,1,4.0,964982703,1985-04-16 09:22:31.500
1,1,3,4.0,964981247,1985-04-16 09:10:23.500
2,1,6,4.0,964982224,1985-04-16 09:18:32.000
3,1,47,5.0,964983815,1985-04-16 09:31:47.500
4,1,50,5.0,964982931,1985-04-16 09:24:25.500
...,...,...,...,...,...
100784,610,166534,4.0,1493848402,1993-09-01 22:56:41.000
100785,610,168248,5.0,1493850091,1993-09-01 23:10:45.500
100786,610,168250,5.0,1494273047,1993-09-04 09:55:23.500
100787,610,168252,5.0,1493846352,1993-09-01 22:39:36.000
